In [ ]:
import json
import time
import hashlib
import hmac
import base64
import uuid

# Declare empty header dictionary
apiHeader = {}
# open token
token = 'id_token' # copy and paste from the SwitchBot app V6.14 or later
# secret key
secret = 'id_secreto' # copy and paste from the SwitchBot app V6.14 or later
nonce = uuid.uuid4()
t = int(round(time.time() * 100000))
string_to_sign = '{}{}{}'.format(token, t, nonce)

string_to_sign = bytes(string_to_sign, 'utf-8')
secret = bytes(secret, 'utf-8')

sign = base64.b64encode(hmac.new(secret, msg=string_to_sign, digestmod=hashlib.sha256).digest())
print ('Authorization: {}'.format(token))
print ('t: {}'.format(t))
print ('sign: {}'.format(str(sign, 'utf-8')))
print ('nonce: {}'.format(nonce))

#Build api header JSON
apiHeader['Authorization']=token
apiHeader['Content-Type']='application/json'
apiHeader['charset']='utf8'
apiHeader['t']=str(t)
apiHeader['sign']=str(sign, 'utf-8')
apiHeader['nonce']=str(nonce)


In [ ]:
import requests
import time
import uuid
import hmac
import hashlib
import base64
import json

def generate_signature(api_token, secret_token):
    t = int(time.time() * 1000)
    nonce = str(uuid.uuid4())
    string_to_sign = f"{api_token}{t}{nonce}".encode('utf-8')
    secret_key = secret_token.encode('utf-8')
    signature = hmac.new(secret_key, string_to_sign, hashlib.sha256).digest()
    sign = base64.b64encode(signature).decode('utf-8').upper()
    return t, nonce, sign

def get_sensor_data(api_token, secret_token, device_id, start_time, end_time):
    url = f"https://api.switch-bot.com/v1.1/devices/{device_id}/historical-data"
    t, nonce, sign = generate_signature(api_token, secret_token)
    date = time.strftime('%Y%m%dT%H%M%SZ', time.gmtime())

    authorization_header = (
        f"HMAC-SHA256 "
        f"Credential={api_token}, "
        f"SignedHeaders=content-type;host;x-amz-date, "
        f"Signature={sign}"
    )

    headers = {
        "Authorization": authorization_header,
        "Content-Type": "application/json",
        "X-Amz-Date": date
    }

    params = {
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code, response.text

api_token = 'id_token'
secret_token = 'id_secreto'
device_id = 'id_dispositivo'
start_time = 1622505600000  # Timestamp in milliseconds
end_time = 1622592000000  # Timestamp in milliseconds

sensor_data = get_sensor_data(api_token, secret_token, device_id, start_time, end_time)

print(json.dumps(sensor_data, indent=4))
